# Mini-Project: Congestion in a Warehouse / Port (Queue Simulator)

This notebook is an optional **mini-project** for the course.

We will build a very simple discrete-time model of **congestion** in a warehouse,
cross-dock, port terminal, or similar facility.

At each time period (for example, one hour or one day):

- Some number of jobs arrive (trucks, containers, orders, etc.).
- The system has a fixed **capacity**: the maximum number of jobs that can be processed.
- If arrivals exceed capacity, the extra jobs wait in a **queue**.
- We track how the queue evolves over time.

We will build the simulator in four stages:

1. **Session 1:** Understand the story and choose data structures.
2. **Session 2:** Implement a simple deterministic queue evolution with loops.
3. **Session 3:** Add randomness and wrap the logic into functions.
4. **Session 4:** Save results to a CSV file and make basic plots.


---
## 1. Session 1 – Understanding the Story (No Code Yet)

### Informal story

Think of a warehouse dock, a cross-dock platform, or a port terminal:

- Trucks / containers / jobs **arrive** over time.
- The facility can process only a limited number per period (capacity).
- Unprocessed jobs wait in a **queue** for the next period.
- If arrivals are often larger than capacity, the queue can grow a lot (congestion).

We will simulate this in **discrete time**:

- At each period:
  - New arrivals come in.
  - The system processes up to `capacity` jobs.
  - We update the number of jobs in the queue.

### Questions to think about

1. How would you store the queue length at each period?  
2. How would you store the number of arrivals per period?  
3. What performance measures could be interesting?
   - maximum queue length,
   - average queue length,
   - number of periods with "severe congestion" (queue above some threshold).

You can discuss this with a partner or write your ideas in a text cell.


> **Optional thought experiment:**  
> Imagine a cross-dock where each hour 0–5 trucks arrive (uniformly),
> and the dock can serve 3 trucks per hour.
> Do you expect the queue to grow without bound, oscillate, or usually stay low?


---
## 2. Session 2 – Core Loop (Deterministic Version)

We begin with a simple **deterministic** version:

- Fixed number of arrivals per period (for now).
- Fixed service capacity per period.
- No randomness yet.
- We just want to build and understand the loop that updates the queue.


In [ ]:
# Parameters

T = 20              # number of periods (e.g., hours or days)
capacity = 3        # max jobs processed per period
arrivals_per_period = 2  # fixed arrivals for now

queue = 0
queue_history = []

print("Period | Arrivals | Served | Queue at end")
print("-----------------------------------------")

for t in range(1, T + 1):
    # New arrivals join the queue
    arrivals = arrivals_per_period
    queue += arrivals

    # Jobs served this period (cannot exceed queue or capacity)
    served = min(queue, capacity)
    queue -= served

    queue_history.append(queue)

    print(f"{t:6d} | {arrivals:8d} | {served:6d} | {queue:13d}")

> **Tasks:**
>
> 1. Run the cell with `arrivals_per_period = 2` and `capacity = 3`.
>    - What happens to the queue?
> 2. Change `arrivals_per_period` to 4 and rerun.
>    - Does the queue grow, shrink, or stabilise?
> 3. Try a pattern where arrivals depend on `t` (time), for example:
>    - small arrivals in the first half of the horizon,
>    - larger arrivals in the second half (a "peak" period).


---
## 3. Session 3 – Random Arrivals and Functions

Now we will:

1. Introduce **random arrivals** using the `random` module.
2. Wrap the per-period logic in a function `one_period`.
3. Write a function `simulate` that runs the system for many periods.


In [ ]:
import random

def random_arrivals():
    """Return a random number of arrivals for one period.

    For now, we choose an integer uniformly from 0 to 5.
    You can change this later if you like.
    """
    return random.randint(0, 5)

In [ ]:
def one_period(queue, capacity):
    """Simulate one period of operation.

    Parameters
    ----------
    queue : int
        Number of jobs waiting at the start of the period.
    capacity : int
        Maximum number of jobs that can be processed in this period.

    Returns
    -------
    new_queue : int
        Queue length at the end of the period.
    arrivals : int
        Jobs that arrived during the period.
    served : int
        Jobs that were processed during the period.
    """
    arrivals = random_arrivals()
    queue += arrivals

    served = min(queue, capacity)
    queue -= served

    return queue, arrivals, served

In [ ]:
def simulate(T=100, capacity=3):
    """Simulate T periods of the queueing system.

    Returns a dictionary with time series for queue length, arrivals, and served jobs.
    """
    queue = 0

    history_queue = []
    history_arrivals = []
    history_served = []

    for t in range(T):
        queue, arrivals, served = one_period(queue, capacity)
        history_queue.append(queue)
        history_arrivals.append(arrivals)
        history_served.append(served)

    return {
        "queue": history_queue,
        "arrivals": history_arrivals,
        "served": history_served,
    }

In [ ]:
# Try the simulator

results = simulate(T=50, capacity=3)

print("Final queue length:", results["queue"][-1])
print("Maximum queue length:", max(results["queue"]))
print("Average queue length:", sum(results["queue"]) / len(results["queue"]))

> **Tasks:**
>
> 1. Run the simulation multiple times and observe how the maximum and average queue
>    lengths vary (they are random now).
> 2. Change the capacity to 2, 4, or 5 and compare the results.
> 3. What seems to happen when capacity is much smaller than the average number of arrivals?


### Optional: Congestion Threshold

Suppose we define **severe congestion** as `queue > 10`.

We can compute the proportion of periods that have severe congestion.


In [ ]:
def proportion_severely_congested(queue_history, threshold=10):
    count = 0
    for q in queue_history:
        if q > threshold:
            count += 1
    return count / len(queue_history)

prop = proportion_severely_congested(results["queue"], threshold=10)
print("Proportion of severely congested periods:", prop)

> **Optional tasks:**
>
> 1. Experiment with different capacity values and thresholds.
> 2. Interpret the proportion of severely congested periods as a kind of **risk measure**.


---
## 4. Session 4 – Saving Results and Plotting

Now we will:

1. Plot the queue length over time using `matplotlib`.
2. Save the simulation results to a CSV file.


In [ ]:
import matplotlib.pyplot as plt

# Run a fresh simulation
results = simulate(T=100, capacity=3)

plt.plot(results["queue"])
plt.xlabel("Period")
plt.ylabel("Queue length")
plt.title("Queue length over time")
plt.show()

In [ ]:
# Explore visually: try a higher capacity

results_cap5 = simulate(T=100, capacity=5)

plt.plot(results["queue"], label="capacity = 3")
plt.plot(results_cap5["queue"], label="capacity = 5")
plt.xlabel("Period")
plt.ylabel("Queue length")
plt.title("Impact of capacity on congestion")
plt.legend()
plt.show()

> **Tasks:**
>
> 1. Plot and compare queue trajectories for different capacities.
> 2. Visually identify when the system seems very congested.
> 3. Try to choose a capacity that seems to keep congestion at a reasonable level.


In [ ]:
# Save results to CSV

results = simulate(T=50, capacity=3)
filename = "queue_simulation_results.csv"

with open(filename, "w") as f:
    f.write("period,queue,arrivals,served\n")
    for t, (q, a, s) in enumerate(
        zip(results["queue"], results["arrivals"], results["served"]), start=1
    ):
        f.write(f"{t},{q},{a},{s}\n")  # noqa: E999

print(f"Wrote results to {filename}")

> In Google Colab, you can download the file using:
>
> ```python
> from google.colab import files
> files.download("queue_simulation_results.csv")
> ```
>
> In a local Jupyter environment, you can inspect the file in your working directory.


---
## 5. Extensions (Optional, For Curious Minds)

Here are some ideas if you want to push the model further:

1. **Time-varying arrivals:**  
   Make arrivals depend on the period (e.g. peak hours with more arrivals).

2. **Time-varying capacity:**  
   Simulate days with reduced capacity, for example due to maintenance or bad weather.

3. **Maximum queue size & lost jobs:**  
   Impose a maximum queue size (e.g. parking lot capacity) and count how many arrivals
   are "lost" when the queue is full.

4. **Multiple replications:**  
   Run many simulations with the same parameters and compare distributions of:
   - maximum queue length,
   - proportion of severely congested periods,
   - final queue length.
